In [49]:
import random, numpy as np
alphap = 2
expon = 2 * alphap - 1
probp = 0.75

In [50]:
def betadist(alpha):
    """gives a random number from beta distribution"""
    return random.betavariate(alpha,alpha)

In [51]:
def decision(probability):
    """
    decides with a given probability whether to keep the right part
    """
    if float(probability) > random.random():
        return True
    else: 
        return False

In [52]:
def splitting(left,right):
    """
    splits a given segment. left and right are endpoints of the segment
    """
    segment = right - left
    xL = segment * betadist(alphap)
    xR = segment - xL
    splitpoint = left + xL
    flag = decision(probp)
    xLp = xL**expon
    xRp = xR**expon
    change = xLp + xRp - segment**expon
    return splitpoint, flag, xLp, xRp, change

In [53]:
def fraclen(List, index):
    sum = 0
    for i in range(len(List[:index+1])):
        sum += List[i]
    return sum

In [54]:
def pickindex(expons, exponsum):
    """picks up a segment to be subsequently split"""
    r = random.uniform(0, exponsum)
    for index in range(len(expons)):
            if fraclen(expons, index) < r: # this call to fraclen consumes time. use np.cumsum() 
                continue
            else:
                return index


def pickindex_v2(expons, exponsum):
    """
    comment : slower than v1
    picks up a segment to be subsequently split
    """
    r = random.uniform(0, exponsum)
    expons_cumulative = np.cumsum(expons) # summing all might not be efficinnt. instead sum only what is needed
    for index in range(len(expons)):
            if expons_cumulative[index] < r:
                continue
            else:
                return index
            
def pickindex_v3(expons, exponsum):
    """
    comment : falser than v2 and v1
    picks up a segment to be subsequently split
    """
    r = random.uniform(0, exponsum)
    sum_ = 0
    for index in range(len(expons)):
        sum_ += expons[index]
        if sum_ < r:
            continue
        else:
            return index

In [87]:
points = [0.,1.]
flags = [True]
expons = [1.] # like probability
exponsum = 1.0 # sum  of probability

for i in range(3):
    # print("********step i=", i)
    index = pickindex_v3(expons, exponsum)
    # print("chosen index = ", index)
    if flags[index] == True:
        left = points[index]
        right = points[index+1]
        splitpoint, flag, xLp, xRp, change = splitting(left,right)
        points.insert(index+1,splitpoint)
        flags.insert(index+1,flag)
        #print("before expons ", expons)
        
#         del expons[index]
#         expons.insert(index,xLp)

        expons[index] = xLp # without deleting element

        expons.insert(index+1,xRp)
        #print("after expons ", expons)
        exponsum += change
        pass
    pass

print(points)
print(flags)
print(expons)
print(len(expons))
print(exponsum)
# print(np.sum(expons))

[0.0, 0.3860680276398328, 0.5034779932908803, 0.6520009304053185, 1.0]
[True, False, True, False]
[0.05754286889793422, 0.0016185081215093456, 0.003276276804340225, 0.042143853973486675]
4
0.10458150779727043


# Testing section

In [56]:
%timeit pickindex(expons, exponsum)

1.88 µs ± 31.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [57]:
%timeit pickindex_v2(expons, exponsum)

5.97 µs ± 400 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [58]:
%timeit pickindex_v3(expons, exponsum)

750 ns ± 8.78 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [59]:
def view(points, flags, expons):
    flag_list = flags + [False]
    expon_list = expons + [0.]
    Norma = np.sum(expon_list)
    print(Norma)
    print(exponsum)
    if (exponsum - Norma) > 1e-6:
        print("Normalization mismatched")
        pass
    print("left point, flag, probability")
    for a,b,c in zip(points, flag_list, expon_list):
        print("[{:10.5}, {:5}, {:10.5}]".format(a, b, c/Norma))
        pass
    pass 

In [60]:
view(points, flags, expons)

0.16905595792737185
0.16905595792737177
left point, flag, probability
[       0.0,     1,   0.025983]
[   0.16377,     1,    0.21705]
[   0.49607,     0,    0.75697]
[       1.0,     0,        0.0]


In [61]:
a = [1,2,3,4,5]
np.cumsum(a)

array([ 1,  3,  6, 10, 15])

In [62]:
b = [1,2,3,4]

In [63]:
b

[1, 2, 3, 4]

In [64]:
del b[0]
b.insert(0, 11)

In [65]:
b

[11, 2, 3, 4]

In [66]:
b[0] = 11

In [67]:
b

[11, 2, 3, 4]